In [1]:
import lib
import numpy as np
from scipy import special
import os

# Hartree Fock computation of the ground state of Quantum Dots

In this Jupyter Notebook we are going to study the groundstate of an N electron Quantum Dot. Our intention is to study how the chemical potential has jumps when shells are added (1s,2s2p). In both cases, the Self-Consistent Field method is used.

### Basis functions
First we write the basis we are going to use, i.e., the 3D eigenfunctions of the harmonic oscillator.

In [2]:
lz = 0.5
lx = 0.75

OMEGA_X = 1/lx**2
OMEGA_Z = 1/lz**2

def Hermite_pol(x, n):
    """
    Evaluates the Hermite polynomial of order n at x.

    Parameters
    ==========
    x : float or np.ndarray
        Values in which to evaluate the Hermite polynomial
    n : int
        Oder of the Hermite polynomial

    Returns
    =======
    float or np.ndarray
    """
    return special.eval_hermite(n, x)

def HO_wf(x, n, omega=OMEGA_X):
    """
    Evaluates the n^th eigenfunction of the Harmonic Oscillator at x.
    The units of x are sqrt(hbar/m*omega). 

    Parameters
    ==========
    x : float or np.ndarray(N)
        Values in which to evaluate eigenfunction
    n : int
        Number of the eigenfunction
    omega : float
        Angular frequency of the Harmonic Oscillator

    Returns
    =======
    float or np.ndarray(N)
    """
    #return omega**0.25 * Hermite_pol(np.sqrt(omega)*x, n) * np.exp(-omega*x**2 / 2) / np.sqrt(2**n * np.math.factorial(n) * np.sqrt(np.pi))
    return omega**0.25 * Hermite_pol(np.sqrt(omega)*x, n) / np.sqrt(2**n * np.math.factorial(n) * np.sqrt(np.pi))

def HO_wf_3D(x, y, z, nx, ny, nz, omega_x=OMEGA_X, omega_y=OMEGA_X, omega_z=OMEGA_Z):
    """
    Evaluates the n^th eigenfunction of the (anisotropic) Harmonic Oscillator at x,y,z.
    The units of x,y,z are sqrt(hbar/m*omega). 

    Parameters
    ==========
    x, y, z : float or np.ndarray(N)
        Position in which to evaluate the wave function
    nx, ny, nz : int
        Number of the eigenfunction for each cartesian coordinate
    omega_x, omega_y, omega_z : float
        Harmonic osciallation constant for each cartesian coordinate

    Returns
    =======
    float or np.ndarray(N)
    """
    return HO_wf(x, nx, omega=omega_x)*HO_wf(y, ny, omega=omega_y)*HO_wf(z, nz, omega=omega_z)

def index_to_q_numbers(k):
    """
    Returns the quantum numbers nx, ny, nz associated with the basis index k

    Parameters
    ----------
    k: int
        Index of the basis from 0 to 13

    Returns
    ----------
    nz ,ny, nz : int 
        Quantum numbers
    """

    q_numbers = np.array([(0,0,0),(0,0,1),(0,1,0),(1,0,0),(0,1,1),(1,0,1),(1,1,0),(1,1,1),(1,1,2),(1,2,1),(2,1,1),(1,2,2),(2,1,2),(2,2,1)])

    return q_numbers[k]

We now write the functions to calculate the integrands of the two-body integrals

In [3]:
def integrand_2(R, p, r, q, s):
    r1 = R[:,0:3]
    r2 = R[:,3:6]
    r12 = np.sqrt(np.sum((r1 - r2)**2, axis=-1))
    
    x1 = R[:,0]
    y1 = R[:,1]
    z1 = R[:,2]
    x2 = R[:,3]
    y2 = R[:,4]
    z2 = R[:,5]
    
    n_p = index_to_q_numbers(p-1)
    n_r = index_to_q_numbers(r-1)
    n_q = index_to_q_numbers(q-1)
    n_s = index_to_q_numbers(s-1)
    
    result = 1/r12
    
    result *= HO_wf_3D(x1, y1, z1, n_p[0], n_p[1], n_p[2])
    result *= HO_wf_3D(x2, y2, z2, n_r[0], n_r[1], n_r[2])
    result *= HO_wf_3D(x1, y1, z1, n_q[0], n_q[1], n_q[2])
    result *= HO_wf_3D(x2, y2, z2, n_s[0], n_s[1], n_s[2])
  
    
    return result

def norm_product(p, r, q, s):
    return (np.pi/OMEGA_X)**2 * np.pi/OMEGA_Z

def two_body_integrand(p, r, q, s):
    f = lambda R: integrand_2(R, p, r, q, s)*norm_product(p, r, q, s)
    
    return f

### One- and two- body integral computations
We first initialize some necessary parameters

In [17]:
N_electrons = 2
N_basis = 8
N_points = 100000
integrals_file = "integrals_QD.npy"

normalized_wf = True

max_iter_SCF = 500
eps_SCF = 10
Delta_SCF = 0

The exact one-body integral function is

In [5]:
def analytical_1(p,q):
    n_p = index_to_q_numbers(p-1)
    I = (p==q)*(OMEGA_X*(n_p[0] + n_p[1] + 1) + OMEGA_Z*(n_p[2] + 0.5))
    
    return I

The covariance function for the exponents of the gaussians is

In [6]:
def f_cov(p, r, q, s):
    
    cov = 0.5*np.diag([1/OMEGA_X, 1/OMEGA_X, 1/OMEGA_Z, 1/OMEGA_X, 1/OMEGA_X, 1/OMEGA_Z])
    
    return cov

We then calculate and store the one- and two-body integrals using lib.py

In [7]:
# One- and Two-body integrals
MC_args = {"f_cov":f_cov, "f_integrand":two_body_integrand, "N_points":N_points}
integrals = lib.integral_master(N_basis)
integrals.calculate(integrals_file, analytical_1, MC_args = MC_args)

Integral file already exists. Not computing the integrals. 


We calculate the overlap matrix S

In [8]:
if not normalized_wf:
    S = np.zeros((4,4))
    for p in range(4):
        for q in range(4):
            S[p][q] = (np.pi/(ALPHA[p]+ALPHA[q]))**(1.5)
    SVAL, SVEC = np.linalg.eigh(S) 
    SVAL_minhalf = (np.diag(SVAL**(-0.5))) 
    X = np.dot(SVEC, np.dot(SVAL_minhalf, np.transpose(SVEC)))
else:
    S = np.eye(N_basis)
# print('S',S)

Finally, we run the Self-Consistent Field for Hartree Fock

In [16]:
lib.SCF(N_electrons, integrals, S, max_iter_SCF, eps_SCF)

E = 6.0444444 | N(SCF) = 1
SCF CONVERGED! E = 6.0444444444


In [14]:

# Este bloque habrá que acabar quitándolo
cov = f_cov(1,1,2,2)
integrand = two_body_integrand(1,1,2,2)

N_points = 100000
dim = cov.shape[0]
for i in range(10):
    steps = np.random.multivariate_normal(np.zeros(dim), cov, N_points) 

    I_avg= np.average(integrand(steps))
    #print(norm_product(1,1,1,1))

    print(I_avg)


0.16247090551927373
0.16133363777993243
0.1545630192515409
0.16071213547189636
0.15171139753118018
0.15790618497393233
0.15932009569130196
0.15616132551176265
0.16023981081168343
0.16036522631838035
